# Initialisation

In [104]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, Subset
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import numpy.random as rd
import os
from pathlib import Path


import librosa
import pandas as pd

## Reproductibility : adding training seed

In [105]:
SEED = 0
rd.seed(SEED)
torch.manual_seed(SEED)

# Dataset
## Loading the genre label of each track

In [106]:
# Loading the information about each track (metadata)
path_of_fma_metadata = "../fma_metadata/tracks.csv"
fma_metadata = pd.read_csv(path_of_fma_metadata, header=[0, 1], index_col=0)
# Flatten columns multi-index
fma_metadata.columns = ['__'.join(col).strip() for col in fma_metadata.columns.values]

# Keeping only the genre column
track_genre_df = fma_metadata.reset_index()[['track_id', 'track__genre_top']]
track_genre_df = track_genre_df.rename(columns={"track__genre_top": "music_genre"})
track_genre_df = track_genre_df.dropna(subset=["music_genre"])


# Function to get the audio file path from track_id
def make_audio_path(track_id):
    folder = f"{track_id // 1000:03d}"
    filename = f"{track_id:06d}.mp3"
    return Path("../fma_small") / folder / filename

track_genre_df["path"] = track_genre_df["track_id"].apply(make_audio_path)

all_valid_files = {
    p.resolve() for p in Path("../fma_small").rglob("*.mp3")
}

# Keep only rows where path exists
track_genre_df["exists"] = track_genre_df["path"].apply(lambda p: p.resolve() in all_valid_files)
track_genre_df = track_genre_df[track_genre_df["exists"]].reset_index(drop=True)


music_genres = track_genre_df["music_genre"].unique().tolist()
nbr_music_genres = len(music_genres)
genre_to_idx = {g: i for i, g in enumerate(music_genres)}
track_genre_df["label"] = track_genre_df["music_genre"].map(genre_to_idx)
print(track_genre_df)


      track_id music_genre                         path  exists  label
0            2     Hip-Hop  ..\fma_small\000\000002.mp3    True      0
1            5     Hip-Hop  ..\fma_small\000\000005.mp3    True      0
2           10         Pop  ..\fma_small\000\000010.mp3    True      1
3          140        Folk  ..\fma_small\000\000140.mp3    True      2
4          141        Folk  ..\fma_small\000\000141.mp3    True      2
...        ...         ...                          ...     ...    ...
7995    154308     Hip-Hop  ..\fma_small\154\154308.mp3    True      0
7996    154309     Hip-Hop  ..\fma_small\154\154309.mp3    True      0
7997    154413         Pop  ..\fma_small\154\154413.mp3    True      1
7998    154414         Pop  ..\fma_small\154\154414.mp3    True      1
7999    155066     Hip-Hop  ..\fma_small\155\155066.mp3    True      0

[8000 rows x 5 columns]


## Preprocessing the data with Librosa

In [107]:
#Precomputing mel-spectrogram and puting in cache
CACHE_DIR = "./mel_cache"
os.makedirs(CACHE_DIR, exist_ok=True)
MAX_LEN = 2600 # To unifirm the length of the mel-spectrograms
 
def compute_and_cache_mel(track_id, path):
    cache_path = f"{CACHE_DIR}/{track_id}.npy"

    if os.path.isfile(cache_path):
        return cache_path  # already cached

    try:
        audio, _ = librosa.load(path, sr=None, mono=True)
        mel = librosa.feature.melspectrogram(y=audio, n_mels=128)
        mel_db = librosa.power_to_db(mel)

        # normalize
        mel_db = (mel_db - mel_db.mean()) / (mel_db.std() + 1e-6)

        T = mel_db.shape[1]
        if T < MAX_LEN:
            mel_db = np.pad(mel_db, ((0, 0), (0, MAX_LEN - T)), mode="constant")
        else:
            mel_db = mel_db[:, :MAX_LEN]

        
        np.save(cache_path, mel_db.astype(np.float32))
        return cache_path

    except Exception:
        return None 

mel_paths = []

for i, row in track_genre_df.iterrows():
    cache_path = compute_and_cache_mel(row["track_id"], row["path"])
    mel_paths.append(cache_path)

track_genre_df["mel_path"] = mel_paths

# keep only rows with valid cached mels
track_genre_df = track_genre_df[track_genre_df["mel_path"].notna()].reset_index(drop=True)

C:\Users\lucie\AppData\Local\Temp\ipykernel_24804\4120235938.py:13: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(path, sr=None, mono=True)
c:\Users\lucie\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [108]:
class MusicGenreDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
        self.genre_to_idx = {g:i for i,g in enumerate(df["music_genre"].unique())}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        mel = np.load(row["mel_path"])
        mel = torch.tensor(mel, dtype=torch.float32).unsqueeze(0)

        label = self.genre_to_idx[row["music_genre"]]
        return mel, label

## Spliting the Dataset into Train/Test subsets

In [109]:
indices = track_genre_df.index[
    track_genre_df["path"].apply(lambda p: os.path.isfile(p))
].tolist()


rd.shuffle(indices)

split = int(0.8 * len(indices))
train_indices = indices[:split]
test_indices = indices[split:]

train_dataset = Subset(MusicGenreDataset(track_genre_df), train_indices)
test_dataset = Subset(MusicGenreDataset(track_genre_df), test_indices)

# Baseline Implementation
## Model : 
Small (2D) CNN with 4 convolutional layers (Conv → ReLU → MaxPool →
Dropout), followed by a fully connected layer and softmax output.

In [110]:
# Our baseline model
class CNN_base(nn.Module):
    def __init__(self, num_classes=nbr_music_genres, dropout=0.25):
        super().__init__()
        channels = [1, 32, 64, 128, 256]
        layers = []
        for i in range(4):
            layers += [
                nn.Conv2d(channels[i], channels[i + 1], kernel_size=3, padding=1),
                nn.BatchNorm2d(channels[i+1]),
                nn.ReLU(),
                nn.MaxPool2d(2),
                nn.Dropout2d(dropout)
            ]
           
        self.features = nn.Sequential(*layers)
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    
    def forward(self, x):
        x = self.features(x)
        x = self.gap(x)   
        x = self.classifier(x)
        return x

 • Loss: Cross-entropy.

 • Optimizer: Adam.

In [111]:
model = CNN_base(num_classes=nbr_music_genres, dropout=0.25)
opt = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()


 • Metric: Test accuracy

In [112]:
def torch_confusion_matrix(labels, preds, num_classes):
    with torch.no_grad():
        k = (labels >= 0) & (labels < num_classes)
        inds = num_classes * labels[k] + preds[k]
        cm = torch.bincount(inds, minlength=num_classes**2).reshape(num_classes, num_classes)
    return cm

In [ ]:
def test_model(model, test_dl, device, writer=None, global_step=None, class_names=None,name=""):
    correct = 0
    total = 0
    losses = []
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for mel,label in test_dl:
            mel = mel.to(device)
            label = label.to(device)
            preds = model(mel)
            correct += (preds.argmax(1) == label).sum().item()
            total += label.size(0)
            losses.append(criterion(preds, label).mean().numpy())
            all_preds.append(preds.argmax(1).cpu())
            all_labels.append(label.cpu())
    if writer is not None and global_step is not None:
        writer.add_scalar(f"{name}/test/accuracy", 100 * correct / total, global_step)
        writer.add_scalar(f"{name}/test/loss", np.mean(losses), global_step)
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)

        num_classes = len(class_names) if class_names else 8
        cm = torch.zeros((8, 8), dtype=torch.int64)
        for t, p in zip(all_labels, all_preds):
            cm[t, p] += 1

        # TensorBoard display
        
        fig = plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=False, cmap="Blues", 
                    xticklabels=class_names if class_names else range(num_classes),
                    yticklabels=class_names if class_names else range(num_classes))
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix")

        writer.add_figure("test/confusion_matrix", fig, global_step)
        plt.close(fig)

    print(f"Test accuracy: {100 * correct / total:.2f}%")

# Run the training loop

Use 
[tensorboard]
( https://docs.pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html) 
to monitor training.

In [114]:
device = torch.device("cpu")

train_dl = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=32)


# log a small batch of images and the model graph (if possible)
imgs_sample, labels_sample = next(iter(train_dl))
imgs_sample = imgs_sample.to(device)
grid = make_grid(imgs_sample[:16], nrow=4, normalize=True, scale_each=True)


In [115]:
# provide a global step counter that you can increment in the training loop if desired
global_step = 0
writer = SummaryWriter(log_dir="./runs/genre_classif")

test_model(model, test_dl, device, global_step=global_step)
for epoch in range(3):
    model.train()
    train_bar = tqdm(train_dl, desc=f"Epoch {epoch+1}")
    for mel, labels in train_dl:
        global_step += 1
        mel, labels = mel.to(device), labels.to(device)

        opt.zero_grad()
        logits = model(mel)
        loss = criterion(logits, labels)
        loss.backward()
        opt.step()

        writer.add_scalar("train/cnn_loss", loss.item(), global_step)
        train_bar.set_postfix({"loss": loss.item()})

    test_model(model, test_dl, device, writer, global_step, class_names=music_genres)


RuntimeError: Expected floating point type for target with class probabilities, got Long